# PRO3515 - PFSP LS


## Dupla 9

### Nomes: 
### Lucas Hideki Takeuchi Okamura NUSP:  9274315
### Thales Arantes Kerche Nunes   NUSP: 10769372


Importando as bibliotecas necessárias

In [8]:
import xlwings as xw
import random

Definindo a classe Job que usaremos no algoritmo

In [2]:
class Job:
    def __init__(self,i,pi,di):
        self.i=i     # número do job, pela ordem de chegada
        self.p=pi     # processing time
        self.p_total=sum(self.p)
        self.d=di     # due date
        self.C=0     # completion time
        self.T=0     # tardiness

### Reutilizamos a função disponibilizada pelo professor, com alguns ajustes para ler o número de máquinas

In [9]:
def leInst(sheet_num):
    pl1 = wb1.sheets[sheet_num]
    m=int(pl1.range('B5').value)     # converter em inteiro
    n=int(pl1.range('B6').value)     # converter em inteiro
    Cmax=int(pl1.range('F5').value)     # converter em inteiro
    tab1=pl1.range('A11').options(expand='table', numbers=int).value     # ler como tabela e inteiros
    print()
    print('n =',n)
    print('m =',m)
    print('dados =',tab1)
    jobs=[]
    for i in range(n):
        pi=tab1[i][1:-5]
        jobs.append(Job(tab1[i][0]-1,pi,0))
    
    return m,n,jobs,Cmax

Também reutilizamos a de gravar, novamente, com alguns ajustes

In [4]:
def gravaSched(i,jobs,Cmax):
    n=len(jobs)
    tab=[]
    for job in jobs:
        a=[job.i+1]+job.p+job.C
        tab.append(a)

    if i==0:
        plan=wb2.sheets[0]
        plan.name='I(1)'
    else:
        plan=wb2.sheets.add('I('+str(i+1)+')',after=i)
        
    plan.range('A5').value=['m',5]
    plan.range('A6').value=['n',n]
    plan.range('E5').value=['Cmax',Cmax]
    plan.range('A10').value=['#','p1','p2', 'p3', 'p4', 'p5','C1','C2','C3','C4','C5']
    plan.range('A11').value=tab

    print()
    print('#','p','C')
    for job in jobs:
        print(job.i, job.p, job.C)
    print('Cmax:',Cmax)

Função que permite calcular o tempo que os jobs se completam

In [7]:
def calcCmax(m,jobs):     # esta função calcula os tempos de conclusão dos jobs nas m máquinas
    n=len(jobs)
    c=[[0 for i in range(m)] for j in range(n)]
    p=[job.p for job in jobs]
    
    # primeiro job
    c[0][0]=p[0][0]     # primeira máquina
    for k in range(1,m):     # próximas máquinas
        c[0][k]=c[0][k-1]+p[0][k]
        
    # próximos jobs
    for j in range(1,n):
        c[j][0]=c[j-1][0]+p[j][0]     # primeira máquina
        for k in range(1,m):     # próximas máquinas
            c[j][k]=max(c[j][k-1],c[j-1][k])+p[j][k]
            
    j=0
    for job in jobs:
        job.C=c[j]
        j+=1
    
    return c[-1][-1]

### Função principal `schedLS`

In [68]:
def schedLS(jobs, bestCmax, m):

    melhoria = True
    aux_jobs = jobs[:]
    while melhoria:
        melhoria = False
        jotas = list(range(len(aux_jobs)))
        # random.shuffle(jotas)

        for j1 in jotas:
            job1 = aux_jobs.pop(j1)
            bestPos = j1

            for j2 in range(len(aux_jobs)+1):
                aux_jobs.insert(j2, job1)
                Cmax = calcCmax(m, aux_jobs)

                if Cmax < bestCmax:
                    bestPos = j2
                    melhoria = True
                    bestCmax = Cmax
                    # break
                aux_jobs.pop(j2)
            # if melhoria:
            #     break
            aux_jobs.insert(bestPos, job1)

    
    return aux_jobs,Cmax

## Principal

Caso base pra comprovar o funcionamento do algoritmo implementado

In [85]:
tab1 = [[6,3],[1,3],[8,6],[2,5],[4,6]]
jobs=[]
for i in range(len(tab1)):
    pi=tab1[i]
    jobs.append(Job(i,pi,0))

jobs, Cmax = schedLS(jobs, 32, 2)

print([job.i+1 for job in jobs])
print([job.p for job in jobs])
print(Cmax)

[2, 4, 5, 3, 1]
[[1, 3], [2, 5], [4, 6], [8, 6], [6, 3]]
24


Abre os dois arquivos que vão ser usados

In [86]:
wb1 = xw.Book("xl09 2 B PFSP Cmax NEH.xlsx")
wb2 = xw.Book()

Teste 1:

In [88]:
m,n,jobs,Cmax=leInst(0)

jobs, Cmax = schedLS(jobs, Cmax, 5)

gravaSched(0, jobs, Cmax)


n = 20
m = 5
dados = [[20, 12, 80, 12, 49, 38, 12, 92, 104, 153, 191], [3, 54, 25, 93, 60, 2, 66, 117, 210, 270, 272], [14, 24, 94, 39, 23, 95, 90, 211, 250, 293, 388], [7, 64, 43, 48, 83, 39, 154, 254, 302, 385, 427], [6, 21, 69, 93, 22, 71, 175, 323, 416, 438, 509], [15, 49, 54, 8, 70, 97, 224, 377, 424, 508, 606], [16, 27, 61, 55, 87, 35, 251, 438, 493, 595, 641], [17, 98, 8, 90, 15, 55, 349, 446, 583, 610, 696], [19, 83, 87, 25, 94, 80, 432, 533, 608, 704, 784], [11, 84, 43, 96, 67, 81, 516, 576, 704, 771, 865], [18, 51, 93, 80, 86, 35, 567, 669, 784, 870, 905], [12, 75, 87, 45, 11, 44, 642, 756, 829, 881, 949], [2, 34, 33, 17, 67, 15, 676, 789, 846, 948, 964], [13, 73, 27, 32, 9, 79, 749, 816, 878, 957, 1043], [1, 58, 60, 88, 14, 46, 807, 876, 966, 980, 1089], [4, 19, 84, 25, 47, 8, 826, 960, 991, 1038, 1097], [8, 37, 43, 18, 46, 13, 863, 1003, 1021, 1084, 1110], [10, 82, 14, 98, 27, 14, 945, 1017, 1119, 1146, 1160], [5, 24, 45, 25, 16, 11, 969, 1062, 1144, 1162, 1173], [9, 26, 5

Teste 2:

In [90]:
m,n,jobs,Cmax=leInst(1)

jobs, Cmax = schedLS(jobs, Cmax, 5)

gravaSched(1, jobs, Cmax)


n = 20
m = 5
dados = [[4, 23, 50, 2, 24, 34, 23, 73, 75, 99, 133], [7, 31, 13, 17, 55, 71, 54, 86, 103, 158, 229], [16, 24, 17, 37, 79, 44, 78, 103, 140, 237, 281], [3, 26, 87, 24, 1, 13, 104, 191, 215, 238, 294], [14, 77, 34, 5, 73, 91, 181, 225, 230, 311, 402], [10, 44, 78, 3, 94, 66, 225, 303, 306, 405, 471], [15, 33, 29, 8, 67, 36, 258, 332, 340, 472, 508], [2, 68, 46, 39, 53, 90, 326, 378, 417, 525, 615], [13, 32, 41, 76, 86, 86, 358, 419, 495, 611, 701], [9, 19, 80, 86, 46, 54, 377, 499, 585, 657, 755], [18, 49, 42, 13, 51, 67, 426, 541, 598, 708, 822], [1, 96, 12, 54, 58, 95, 522, 553, 652, 766, 917], [8, 63, 76, 77, 70, 95, 585, 661, 738, 836, 1012], [12, 49, 78, 63, 52, 28, 634, 739, 802, 888, 1040], [20, 98, 67, 13, 55, 57, 732, 806, 819, 943, 1097], [19, 84, 90, 27, 85, 86, 816, 906, 933, 1028, 1183], [5, 66, 63, 16, 90, 9, 882, 969, 985, 1118, 1192], [17, 85, 92, 21, 54, 7, 967, 1061, 1082, 1172, 1199], [6, 83, 38, 41, 21, 34, 1050, 1099, 1140, 1193, 1233], [11, 89, 58, 17

Teste 3:

In [91]:
m,n,jobs,Cmax=leInst(2)

jobs, Cmax = schedLS(jobs, Cmax, 5)

gravaSched(2, jobs, Cmax)


n = 20
m = 5
dados = [[20, 15, 15, 24, 85, 83, 15, 30, 54, 139, 222], [7, 7, 12, 58, 89, 86, 22, 42, 112, 228, 314], [10, 30, 24, 7, 70, 18, 52, 76, 119, 298, 332], [1, 40, 11, 79, 15, 64, 92, 103, 198, 313, 396], [13, 32, 10, 99, 78, 85, 124, 134, 297, 391, 481], [12, 28, 42, 80, 74, 8, 152, 194, 377, 465, 489], [9, 93, 28, 34, 9, 95, 245, 273, 411, 474, 584], [15, 48, 49, 53, 35, 23, 293, 342, 464, 509, 607], [11, 37, 46, 4, 89, 68, 330, 388, 468, 598, 675], [19, 10, 8, 48, 61, 13, 340, 396, 516, 659, 688], [3, 33, 25, 68, 7, 17, 373, 421, 584, 666, 705], [16, 83, 51, 11, 26, 95, 456, 507, 595, 692, 800], [5, 44, 70, 75, 59, 38, 500, 577, 670, 751, 838], [18, 87, 41, 64, 10, 91, 587, 628, 734, 761, 929], [4, 50, 80, 44, 86, 63, 637, 717, 778, 864, 992], [2, 99, 23, 86, 56, 47, 736, 759, 864, 920, 1039], [14, 12, 15, 81, 3, 3, 748, 774, 945, 948, 1042], [17, 91, 79, 25, 47, 17, 839, 918, 970, 1017, 1059], [6, 92, 20, 40, 24, 80, 931, 951, 1010, 1041, 1139], [8, 17, 22, 35, 24, 8, 948

Teste 4:

In [92]:
m,n,jobs,Cmax=leInst(3)

jobs, Cmax = schedLS(jobs, Cmax, 5)

gravaSched(3, jobs, Cmax)


n = 20
m = 5
dados = [[4, 12, 27, 59, 6, 93, 12, 39, 98, 104, 197], [18, 31, 40, 53, 7, 58, 43, 83, 151, 158, 255], [12, 21, 56, 23, 48, 25, 64, 139, 174, 222, 280], [6, 61, 27, 18, 12, 56, 125, 166, 192, 234, 336], [16, 37, 1, 50, 87, 83, 162, 167, 242, 329, 419], [9, 9, 19, 43, 77, 8, 171, 190, 285, 406, 427], [3, 7, 13, 88, 6, 48, 178, 203, 373, 412, 475], [5, 63, 4, 42, 57, 24, 241, 245, 415, 472, 499], [19, 53, 76, 21, 5, 81, 294, 370, 436, 477, 580], [20, 94, 5, 28, 39, 46, 388, 393, 464, 516, 626], [13, 7, 58, 28, 92, 93, 395, 453, 492, 608, 719], [17, 50, 6, 95, 62, 20, 445, 459, 587, 670, 739], [1, 46, 64, 7, 43, 98, 491, 555, 594, 713, 837], [14, 64, 90, 9, 74, 20, 555, 645, 654, 787, 857], [2, 55, 36, 70, 49, 97, 610, 681, 751, 836, 954], [15, 17, 34, 5, 37, 12, 627, 715, 756, 873, 966], [10, 73, 54, 33, 92, 67, 700, 769, 802, 965, 1033], [11, 98, 90, 74, 24, 94, 798, 888, 962, 989, 1127], [8, 6, 2, 20, 91, 18, 804, 890, 982, 1080, 1145], [7, 80, 85, 10, 8, 2, 884, 975, 992

Teste 5:

In [93]:
m,n,jobs,Cmax=leInst(4)

jobs, Cmax = schedLS(jobs, Cmax, 5)

gravaSched(4, jobs, Cmax)


n = 20
m = 5
dados = [[16, 8, 21, 4, 38, 35, 8, 29, 33, 71, 106], [10, 6, 24, 25, 76, 65, 14, 53, 78, 154, 219], [4, 13, 15, 46, 59, 52, 27, 68, 124, 213, 271], [5, 50, 48, 7, 16, 47, 77, 125, 132, 229, 318], [1, 46, 28, 44, 49, 38, 123, 153, 197, 278, 356], [7, 27, 91, 10, 95, 11, 150, 244, 254, 373, 384], [15, 30, 43, 1, 5, 8, 180, 287, 288, 378, 392], [20, 44, 30, 61, 99, 17, 224, 317, 378, 477, 494], [14, 32, 52, 67, 71, 29, 256, 369, 445, 548, 577], [6, 82, 7, 45, 53, 74, 338, 376, 490, 601, 675], [8, 54, 21, 60, 95, 59, 392, 413, 550, 696, 755], [18, 54, 75, 74, 61, 78, 446, 521, 624, 757, 835], [12, 66, 79, 16, 80, 91, 512, 600, 640, 837, 928], [19, 92, 83, 82, 59, 83, 604, 687, 769, 896, 1011], [11, 84, 75, 82, 84, 39, 688, 763, 851, 980, 1050], [3, 98, 93, 55, 47, 55, 786, 879, 934, 1027, 1105], [2, 94, 15, 6, 95, 79, 880, 895, 940, 1122, 1201], [9, 86, 67, 62, 85, 20, 966, 1033, 1095, 1207, 1227], [13, 75, 64, 46, 23, 19, 1041, 1105, 1151, 1230, 1249], [17, 21, 57, 80, 8, 2,

Teste 6:

In [94]:
m,n,jobs,Cmax=leInst(5)

jobs, Cmax = schedLS(jobs, Cmax, 5)

gravaSched(5, jobs, Cmax)


n = 20
m = 5
dados = [[4, 9, 38, 3, 25, 33, 9, 47, 50, 75, 108], [14, 40, 27, 13, 17, 71, 49, 76, 89, 106, 179], [3, 64, 2, 2, 63, 51, 113, 115, 117, 180, 231], [13, 3, 26, 73, 45, 58, 116, 142, 215, 260, 318], [8, 33, 71, 30, 23, 17, 149, 220, 250, 283, 335], [17, 30, 20, 88, 34, 26, 179, 240, 338, 372, 398], [16, 36, 65, 95, 12, 30, 215, 305, 433, 445, 475], [11, 69, 10, 60, 42, 59, 284, 315, 493, 535, 594], [2, 47, 91, 70, 45, 23, 331, 422, 563, 608, 631], [1, 77, 6, 35, 78, 68, 408, 428, 598, 686, 754], [20, 33, 81, 69, 7, 46, 441, 522, 667, 693, 800], [10, 74, 22, 52, 97, 41, 515, 544, 719, 816, 857], [9, 63, 94, 70, 47, 93, 578, 672, 789, 863, 956], [19, 98, 39, 39, 49, 77, 676, 715, 828, 912, 1033], [15, 72, 79, 36, 99, 33, 748, 827, 864, 1011, 1066], [5, 30, 3, 95, 9, 53, 778, 830, 959, 1020, 1119], [7, 83, 25, 67, 70, 13, 861, 886, 1026, 1096, 1132], [18, 85, 97, 34, 18, 24, 946, 1043, 1077, 1114, 1156], [6, 85, 22, 83, 1, 10, 1031, 1065, 1160, 1161, 1171], [12, 63, 32, 11, 2

Teste 7:

In [95]:
m,n,jobs,Cmax=leInst(6)

jobs, Cmax = schedLS(jobs, Cmax, 5)

gravaSched(6, jobs, Cmax)


n = 20
m = 5
dados = [[2, 42, 5, 64, 27, 26, 42, 47, 111, 138, 164], [13, 17, 25, 32, 86, 76, 59, 84, 143, 229, 305], [10, 45, 4, 95, 31, 35, 104, 108, 238, 269, 340], [12, 21, 65, 39, 61, 57, 125, 190, 277, 338, 397], [6, 4, 22, 77, 5, 12, 129, 212, 354, 359, 409], [11, 34, 45, 18, 68, 59, 163, 257, 372, 440, 499], [14, 58, 96, 15, 60, 36, 221, 353, 387, 500, 536], [17, 66, 29, 91, 46, 93, 287, 382, 478, 546, 639], [9, 96, 64, 54, 30, 98, 383, 447, 532, 576, 737], [5, 77, 47, 91, 96, 90, 460, 507, 623, 719, 827], [8, 71, 83, 69, 51, 84, 531, 614, 692, 770, 911], [20, 84, 63, 71, 25, 57, 615, 678, 763, 795, 968], [16, 71, 83, 84, 63, 59, 686, 769, 853, 916, 1027], [1, 93, 82, 81, 41, 81, 779, 861, 942, 983, 1108], [18, 95, 73, 64, 50, 14, 874, 947, 1011, 1061, 1122], [7, 81, 61, 46, 19, 33, 955, 1016, 1062, 1081, 1155], [15, 46, 86, 1, 1, 44, 1001, 1102, 1103, 1104, 1199], [4, 74, 42, 4, 7, 47, 1075, 1144, 1148, 1155, 1246], [3, 77, 16, 47, 11, 16, 1152, 1168, 1215, 1226, 1262], [19, 

Teste 8:

In [96]:
m,n,jobs,Cmax=leInst(7)

jobs, Cmax = schedLS(jobs, Cmax, 5)

gravaSched(7, jobs, Cmax)


n = 20
m = 5
dados = [[1, 21, 15, 23, 42, 83, 21, 36, 59, 101, 184], [11, 8, 2, 4, 10, 4, 29, 38, 63, 111, 188], [7, 14, 1, 53, 27, 9, 43, 44, 116, 143, 197], [14, 56, 10, 25, 28, 96, 99, 109, 141, 171, 293], [9, 12, 71, 22, 78, 31, 111, 182, 204, 282, 324], [18, 51, 22, 36, 9, 37, 162, 204, 240, 291, 361], [19, 51, 51, 9, 19, 98, 213, 264, 273, 310, 459], [10, 23, 12, 51, 96, 98, 236, 276, 327, 423, 557], [2, 35, 22, 14, 89, 15, 271, 298, 341, 512, 572], [17, 12, 46, 86, 14, 31, 283, 344, 430, 526, 603], [4, 20, 99, 30, 60, 54, 303, 443, 473, 586, 657], [8, 66, 8, 3, 49, 31, 369, 451, 476, 635, 688], [16, 32, 1, 94, 43, 17, 401, 452, 570, 678, 705], [12, 70, 39, 67, 18, 23, 471, 510, 637, 696, 728], [3, 51, 79, 7, 8, 99, 522, 601, 644, 704, 827], [20, 51, 34, 2, 75, 99, 573, 635, 646, 779, 926], [6, 40, 50, 93, 98, 67, 613, 685, 778, 877, 993], [13, 64, 92, 74, 61, 76, 677, 777, 852, 938, 1069], [5, 75, 50, 38, 75, 73, 752, 827, 890, 1013, 1142], [15, 79, 92, 89, 98, 47, 831, 923, 10

Teste 9:

In [97]:
m,n,jobs,Cmax=leInst(8)

jobs, Cmax = schedLS(jobs, Cmax, 5)

gravaSched(8, jobs, Cmax)


n = 20
m = 5
dados = [[6, 4, 24, 7, 79, 41, 4, 28, 35, 114, 155], [13, 18, 9, 17, 41, 38, 22, 37, 54, 155, 193], [17, 45, 22, 14, 94, 16, 67, 89, 103, 249, 265], [3, 26, 4, 65, 62, 59, 93, 97, 168, 311, 370], [11, 44, 62, 63, 80, 64, 137, 199, 262, 391, 455], [8, 52, 71, 71, 88, 87, 189, 270, 341, 479, 566], [10, 43, 17, 12, 77, 16, 232, 287, 353, 556, 582], [15, 16, 84, 21, 53, 88, 248, 371, 392, 609, 697], [18, 80, 5, 89, 28, 69, 328, 376, 481, 637, 766], [2, 69, 99, 84, 96, 95, 397, 496, 580, 733, 861], [20, 71, 71, 57, 35, 44, 468, 567, 637, 768, 905], [14, 91, 92, 13, 74, 80, 559, 659, 672, 842, 985], [12, 96, 49, 77, 71, 89, 655, 708, 785, 913, 1074], [1, 66, 73, 83, 96, 24, 721, 794, 877, 1009, 1098], [5, 55, 20, 45, 74, 31, 776, 814, 922, 1083, 1129], [16, 32, 93, 65, 30, 25, 808, 907, 987, 1113, 1154], [7, 57, 71, 59, 4, 46, 865, 978, 1046, 1117, 1200], [9, 94, 55, 18, 70, 25, 959, 1033, 1064, 1187, 1225], [4, 92, 14, 5, 40, 5, 1051, 1065, 1070, 1227, 1232], [19, 30, 26, 73, 

Teste 10:

In [98]:
m,n,jobs,Cmax=leInst(9)

jobs, Cmax = schedLS(jobs, Cmax, 5)

gravaSched(9, jobs, Cmax)


n = 20
m = 5
dados = [[18, 11, 92, 34, 3, 10, 11, 103, 137, 140, 150], [2, 15, 26, 3, 1, 56, 26, 129, 140, 141, 206], [17, 41, 17, 2, 46, 5, 67, 146, 148, 194, 211], [9, 21, 44, 61, 82, 38, 88, 190, 251, 333, 371], [14, 19, 72, 16, 40, 72, 107, 262, 278, 373, 445], [11, 49, 22, 75, 39, 36, 156, 284, 359, 412, 481], [13, 40, 37, 41, 60, 72, 196, 321, 400, 472, 553], [20, 46, 75, 83, 3, 96, 242, 396, 483, 486, 649], [6, 45, 71, 86, 89, 37, 287, 467, 569, 658, 695], [19, 65, 99, 56, 10, 87, 352, 566, 625, 668, 782], [8, 82, 31, 96, 47, 73, 434, 597, 721, 768, 855], [4, 53, 72, 53, 39, 44, 487, 669, 774, 813, 899], [7, 45, 95, 47, 94, 5, 532, 764, 821, 915, 920], [5, 57, 57, 91, 80, 13, 589, 821, 912, 995, 1008], [12, 73, 81, 44, 14, 33, 662, 902, 956, 1009, 1042], [3, 88, 54, 36, 41, 17, 750, 956, 992, 1050, 1067], [15, 46, 44, 58, 21, 26, 796, 1000, 1058, 1079, 1105], [16, 7, 39, 24, 54, 14, 803, 1039, 1082, 1136, 1150], [10, 57, 31, 51, 24, 1, 860, 1070, 1133, 1160, 1161], [1, 69, 81, 

Fecha o workbook de Dados

In [99]:
wb1.close()